In [1]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm

# Original Parameter values

## ODE

In [2]:
def f(x, t):
    
    x_1,x_2,x_3,x_4,x_5 = x 
   
    dx_1 = λ_1 -(λ_2 * λ_3 * x_5 + λ_4 + λ_5)*x_1 + λ_6*x_2
    dx_2 = λ_2 * λ_3 * x_5*(x_1 + δ*x_4)-(λ_6+λ_5+λ_7)*x_2
    dx_3 = λ_7*x_2 -(λ_8 + λ_5+λ_9)*x_3 
    dx_4 = λ_9*x_3 + λ_4*x_1 -(λ_5+λ_2*λ_3*δ*x_5)*x_4 
        
    dx_5 = λ_10*x_3*(1-x_5)/λ_3 - λ_11*x_5
    
    return [dx_1,dx_2,dx_3,dx_4,dx_5]



## Initial values

In [3]:
x_2 = 0
x_3 = 0
x_4 = 0
x_5 = 0.05

x_1 = 1  - x_2 - x_3 - x_4

In [4]:
λ_h = 128227/(365*13.1e6) #2.68e-5
λ_t = (1755278781*2)/(365*2.063e10) #4.67e-4

μ_h = 134313/(365*13.1e6) #2.809e-5
μ_e = 2/445 # 0.004
μ_t = 1/(4*365) # 0.00068

r_1 = 1/14 # 0.07
r_2 = 1/14 # 0.07

β_1 = 0.3*1/365 #8.2e-4  #0.3 
β_dog = 2*0.3*1/365
β_2 = 1/365 # 0.0027
γ_h = 1/10.5 # 0.095

ψ = 2.699e6/(13.1e6*365) #0.000564
δ = 0.03 

N_h = 13.1e6
N_t= 2.063e10


λ_1= λ_h/N_h
λ_2= β_1
λ_3= N_t/N_h
λ_4= ψ
λ_5= μ_h 
λ_6= r_2
λ_7= γ_h
λ_8= μ_e
λ_9= r_1
λ_10= β_2
λ_11= μ_t


In [5]:
params = np.array([λ_1, λ_2,λ_3,λ_4,λ_5,λ_6,λ_7,λ_8,λ_9,λ_10,λ_11])
string_params = ['λ_1', 'λ_2', 'λ_3','λ_4','λ_5','λ_6','λ_7','λ_8','λ_9','λ_10','λ_11']

In [6]:
x_1 = 1 - x_2 - x_3 - x_4


x0 = [x_1, x_2, x_3, x_4,    #humans
      x_5]  #ticks

ts = np.linspace(0,183, 183)

In [7]:
original = odeint(f,x0,ts)

## Repeatedly Solve ODE with perturbed Parameters

In [8]:
def perturb_params(n, original_params, scale):
    
    solutions = []
    ts = np.linspace(0,183, 183)

    for i in tqdm(range(n), leave = False):
        perturbed_params = np.random.uniform( (1-scale)*original_params, (1+scale)*original_params)
        
        λ_1, λ_2,λ_3,λ_4,λ_5,λ_6,λ_7,λ_8,λ_9,λ_10,λ_11 = perturbed_params
        x_1 = 1 - x_2 - x_3 - x_4
        
        x0 = [x_1, x_2, x_3, x_4,    #humans
              x_5]  #ticks

        ts = np.linspace(0,183, 183)

        #define function again to get new params
        def f(x, t):
            x_1,x_2,x_3,x_4,x_5, = x 
   
            dx_1 = λ_1 -(λ_2 * λ_3 * x_5 + λ_4 + λ_5)*x_1 + λ_6*x_2
            dx_2 = λ_2 * λ_3 * x_5*(x_1 + δ*x_4)-(λ_6+λ_5+λ_7)*x_2
            dx_3 = λ_7*x_2 -(λ_8 + λ_5+λ_9)*x_3 
            dx_4 = λ_9*x_3 + λ_4*x_1 -(λ_5+λ_2*λ_3*δ*x_5)*x_4 
        
            dx_5 = λ_10*x_3*(1-x_5)/λ_3 - λ_11*x_5
            return [dx_1,dx_2,dx_3,dx_4,dx_5]

    solutions.append(odeint(f, x0, ts))
        
    return solutions

## Functions for finding the minimum and maximum deviations

In [9]:
def get_min_max_2(solutions):
    sol_arr = np.array(solutions)
    #get min/max solutions
    S_max = np.amax(sol_arr, axis=0)
    S_min = np.amin(sol_arr, axis=0)
    #get index of min/max solutions, index is taken from the first entry, if multiple are available
    S_max_index = np.argmax(sol_arr, axis=0)
    S_min_index = np.argmin(sol_arr, axis=0)

    return [S_max, S_min, S_max_index, S_min_index]

In [10]:
def find_max_min_diviation_param(sol_max, sol_min, original_curves, index_max, index_min, changed_params):
    save_max_param_per_compartment = []
    save_min_param_per_compartment = []

    k, n = sol_max.shape
    for i in range(n):
        ind_max = np.argmax(sol_max[:,i]-original_curves[:,i])
        ind_min = np.argmin(sol_min[:,i]-original_curves[:,i]) #takes the most negative one
        save_max_param_per_compartment.append(changed_params[index_max[ind_max,i]])
        save_min_param_per_compartment.append(changed_params[index_min[ind_min,i]])
    return [save_max_param_per_compartment, save_min_param_per_compartment]

In [11]:
def perturb_single_param_2(n, original_params, scale, index):

    mesh = np.linspace(original_params[index]*(1-scale), original_params[index]*(1+scale), n)
    solutions = []
    parameter_diviations = []
    ts = np.linspace(0,183, 183)

    for i in tqdm(range(n)):
        perturbed_params = np.copy(original_params)
        perturbed_params[index] = mesh[i]
        parameter_diviations.append(mesh[i])
        λ_1, λ_2,λ_3,λ_4,λ_5,λ_6,λ_7,λ_8,λ_9,λ_10,λ_11 = perturbed_params
        x_1 = 1 - x_2 - x_3 - x_4


        
        x0 = [x_1, x_2, x_3, x_4,    #humans
              x_5]  #ticks
        #define function again to get new params
        def f(x, t):
            x_1,x_2,x_3,x_4,x_5, = x 
   
            dx_1 = λ_1 -(λ_2 * λ_3 * x_5 + λ_4 + λ_5)*x_1 + λ_6*x_2
            dx_2 = λ_2 * λ_3 * x_5*(x_1 + δ*x_4)-(λ_6+λ_5+λ_7)*x_2
            dx_3 = λ_7*x_2 -(λ_8 + λ_5+λ_9)*x_3 
            dx_4 = λ_9*x_3 + λ_4*x_1 -(λ_5+λ_2*λ_3*δ*x_5)*x_4 
        
            dx_5 = λ_10*x_3*(1-x_5)/λ_3 - λ_11*x_5
    
            return [dx_1,dx_2,dx_3,dx_4,dx_5]
        solutions.append(odeint(f, x0, ts))

    return solutions, parameter_diviations

# Functions for an automatic plot

In [12]:
def plot_compartments_2(original_curves, perturbed_params,  single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['x_1', 'x_2', 'x_3', 'x_4',    #humans
                         'x_5'] #x_6 = S_h
    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)
    max_params_per_comp, min_params_per_comp = find_max_min_diviation_param(S_max, S_min, original, S_max_index, S_min_index, perturbed_params)


    #Epidemie
    figure, axis = plt.subplots(4, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(4):
        axis[i].plot(ts, S_max[:,i], color = 'r')
        axis[i].plot(ts, original_curves[:,i], color = 'g')
        axis[i].plot(ts, S_min[:,i], color = 'b')
        axis[i].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/epidemic_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/epidemic_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)

    plt.close(figure)
    plt.close('all')


    figure, axis = plt.subplots(2, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(4,5):
        axis[i-4].plot(ts, S_max[:,i], color = 'r')
        axis[i-4].plot(ts, original_curves[:,i], color = 'g')
        axis[i-4].plot(ts, S_min[:,i], color = 'b')
        axis[i-4].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i-4].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/tick_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/tick_plot_' + string_params[index] +
                       '_perturbed.jpg', bbox_inches = 'tight', dpi=500)
    plt.close(figure)
    plt.close('all')

In [13]:
def plot_compartments_all(original_curves, single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['x_1', 'x_2', 'x_3', 'x_4',    #humans
                         'x_5'] # x_6=S_h

    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)

    #Epidemie
    figure, axis = plt.subplots(4, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(4):
        axis[i].plot(ts, S_max[:,i], color = 'r')
        axis[i].plot(ts, original_curves[:,i], color = 'g')
        axis[i].plot(ts, S_min[:,i], color = 'b')
        axis[i].legend(['max','original','min'])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/epidemic_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')

 
    figure, axis = plt.subplots(2, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(4,5):
        axis[i-4].plot(ts, S_max[:,i], color = 'r')
        axis[i-4].plot(ts, original_curves[:,i], color = 'g')
        axis[i-4].plot(ts, S_min[:,i], color = 'b')
        axis[i-4].legend(['max','original','min'])
        axis[i-4].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/tick_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')


    #Only for the report

    plt.figure(figsize=(8.46, 4.7))
    plt.plot(ts, S_max[:,3], color = 'r')
    plt.plot(ts, original_curves[:,3], color = 'g')
    plt.plot(ts, S_min[:,i], color = 'b')
    plt.xlabel("t in days")
    plt.title("Uncertainty of compartment $Q_2$",fontsize = 16)
    plt.legend(['max','original','min'])
    plt.savefig('perturbed_compartment_plots/Simulation_uncertainty_all_perturbed_Q_2.jpg', bbox_inches = 'tight', dpi = 500)
    plt.close(figure)
    plt.close('all')

# Functions for calculating and ordering the two uncertainty measures

In [14]:
def find_most_uncertain_parameter(list_of_matrices,list_of_parameters):
    n = len(list_of_matrices) #this corresponds to the number of parameters


    #Calculate uncertainty by adding up all differences of the min and max curves
    vector_rowSum = []
    sum_all_compartments = []
    for i in range(n):
        A = np.matrix(np.copy(list_of_matrices[i]))
        v = A.sum(axis=0)
        deviation_all_compartments = np.sum(v)
        vector_rowSum.append(deviation_all_compartments)
        sum_all_compartments.append(v)
    #we want a list of all parameters and their uncertainty value
    ordered_by_uncertainty = []
    #if the largest value is recorded, then the value is set to -1, to get the second biggest value.
    for i in range(len(vector_rowSum)):
        k = np.argmax(vector_rowSum)
        value, name = vector_rowSum[k]/183, list_of_parameters[k]
        ordered_by_uncertainty.append([name, value])
        #the second largest. All values in these matrices are positive, so we simply put it to -1
        vector_rowSum[k] = -1


    return ordered_by_uncertainty

## Plot all curves and calculate the sensitivity

## You need a folder with the name: "perturbed_compartment_plots" in your path

In [15]:
def plot_all_uncertainty_with_params(original_curves, params, string_params, n_perturb, perturb_scale):

    list_of_maxMinusmin = []
    for i in range(len(params)):
        single_perturbed_sol, parameter_perturbed = perturb_single_param_2(n_perturb, params, perturb_scale, i)
        S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_perturbed_sol)
        list_of_maxMinusmin.append(S_max-S_min)
        plot_compartments_2(original_curves, parameter_perturbed, single_perturbed_sol, string_params, i)
        print('Done with parameter ' + string_params[i])

    #calculate the min/ max diviation quantitatively
    print('Ordered from largest to smallest influence overall')
    all_params_ordered_by_uncertainty = find_most_uncertain_parameter(list_of_maxMinusmin, string_params)
    df = pd.DataFrame(all_params_ordered_by_uncertainty, columns=['param_name', 'value'])
    df.to_csv('all_params_ordered_by_uncertainty.csv')
    print(df)


    #add plot: all params are perturbed
    all_perturbed = perturb_params(n_perturb, params, perturb_scale)
    plot_compartments_all(original_curves,  all_perturbed, ['all'],0)
    print('All parameters were perturbed. The computation is done.')

## Perform perturbation and uncertainty analysis

The plots are saved in "perturbed_compartment_plots". 

The results of the uncertainty analysis is stored in two files:

all_params_ordered_by_uncertainty.csv
all_params_ordered_by_uncertainty_influence_on_I.csv

In [17]:
plot_all_uncertainty_with_params(original, params, string_params,100000, 0.05)

  0%|          | 0/100000 [00:00<?, ?it/s]

Done with parameter λ_1


  0%|          | 0/100000 [00:00<?, ?it/s]

Done with parameter λ_2


  0%|          | 0/100000 [00:00<?, ?it/s]

...

Done with parameter λ_10


  0%|          | 0/100000 [00:00<?, ?it/s]

Done with parameter λ_11
Ordered from largest to smallest influence overall
   param_name         value
0         λ_2  2.899817e-02
1         λ_3  2.899606e-02
2         λ_7  2.687382e-02
3         λ_9  1.828604e-02
4         λ_6  1.537835e-02
5         λ_8  4.255874e-03
6        λ_11  9.286249e-04
7         λ_4  8.551079e-04
8         λ_5  2.416570e-04
9        λ_10  5.350133e-06
10        λ_1  1.807607e-11


  0%|          | 0/100000 [00:00<?, ?it/s]

All parameters were perturbed. The computation is done.
